# Carga de ficheros y preprocesamiento de datos

### Si se prueba en local usando docker-compose, hay que hacer un chmod 777 -R . sobre la carpeta notebooks, es un volumen compartido para docker.

Los datos de tráfico se supone que están en /mnt/volume/cvs_ok. Este volumen se utiliza también para descargar los datos de puntos de medida y los festivos, en GCP hay que crear un filestorage persistente.



In [40]:
## Suprimimos warnings

import warnings
warnings.filterwarnings('ignore')


In [2]:
import pyspark
import numpy as np
import pandas as pd
import sys
import os

print (pyspark.__version__)
print (np.__version__)
print (pd.__version__)
print (sys.version)

4.0.0
1.26.0
2.3.2
3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [3]:
import urllib.request

DescargarFicheros = False

# Activar para volver a descargar los puntos de medición. Los descarga en el volumen compartido
# En despliegue de GCP añadir un filestorage y montarlo.


def DescargaPuntos():
  for q in range(1, 400, 1):
    url = "https://datos.madrid.es/egob/catalogo/202468-" + str(q) + "-intensidad-trafico.csv"
    output_file = "/mnt/volume/puntos/" + str(q) + ".csv"
    try:
      urllib.request.urlretrieve(url, output_file)
    except Exception as e:
      pass

if (DescargarFicheros):
  DescargaPuntos()

In [4]:
entornos = []
#local = ('local[*]','AppCLusterLocal','/Users/Juanma/Desktop/work-consolidado/01-2024.csv')
local = ('local[*]','AppCLusterLocal','/media/juanma/spark/datos/01-2024.csv')
compose = ('spark://master:7077','AppCLusterTFM','/mnt/volume/cvs_ok/01-2024.csv')
entornos.append(local)
entornos.append(compose)
fichero = ''
entornos


from pyspark.sql import SparkSession
import pandas as pd

global spark

def ConexionCluster(entorno=0):
# jobs: http://localhost:4040/jobs/
    global fichero    
    srv = entornos[entorno][0]
    app = entornos[entorno][1]
    fichero = entornos[entorno][2]    
    print (srv)
    print (app)
    print (fichero)
    print ('entorno', entorno)
    if entorno > 0:        
        return SparkSession.builder \
        .appName(app) \
        .master(srv) \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.executor.cores", 2) \
        .config("spark.cores.max", 16) \
        .config("spark.driver.maxResultSize", "4g") \
        .getOrCreate()
        
        # esto ya está en el fichero de configuración de spark, no es neceario
        #.config("spark.hadoop.fs.defaultFS", "hdfs://master:9000") \
        
        #.config("spark.local.dir", "/mnt/volume/spark-tmp") \
    else:
        return SparkSession.builder \
            .appName(app) \
            .master(srv) \
            .config("spark.local.dir", "/mnt/volume/spark-tmp") \
            .getOrCreate()        

#    .config("spark.pyspark.python", "/usr/bin/python3.11") \
#    .config("spark.pyspark.driver.python", "/usr/bin/python3.11") \

def CargaMuestra():
    cad = "file:" + fichero
    print (cad)
    return spark.read.format("csv").option("inferSchema","true").option("sep",";").option("header","true") \
    .load(cad)



'''spark = (
    SparkSession.builder
    .appName("ColabSpark4.0.0")   
    .config("spark.driver.memory", "12g")   
    .config("spark.driver.maxResultSize", "2g")  
    # .config("spark.executor.memory", "2g")
    # Add Delta Lake package configuration
    #.config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    #.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)
'''
''''''


''

## Conectamos al cluster y hacemos algunas pruebas

In [5]:

spark = ConexionCluster(1)
df = CargaMuestra()

df.show(3)
# =========================================
# Pruebas...
# =========================================
df = spark.createDataFrame(
    [(1, "Málaga"), (2, "Madrid"), (3, "Cádiz")],
    ["id", "ciudad"]
)
df.show()


print("Spark Context Master:", spark.sparkContext.master)

# Test simple: contar palabras
rdd = spark.sparkContext.parallelize(["hola mundo", "esto es una prueba", "spark con docker"])
word_counts = rdd.flatMap(lambda line: line.split(" ")) \
                 .map(lambda word: (word, 1)) \
                 .reduceByKey(lambda a, b: a + b)

print("Resultados:")
print(word_counts.collect())

spark.conf.set("spark.sql.files.maxPartitionBytes", 134217728)  
spark.conf.set("spark.sql.shuffle.partitions", 8)               

spark.version

spark://master:7077
AppCLusterTFM
/mnt/volume/cvs_ok/01-2024.csv
entorno 1


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/08 20:24:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


file:/mnt/volume/cvs_ok/01-2024.csv


+----+-------------------+---------+----------+---------+-----+----+-----+-------------------+
|  id|              fecha|tipo_elem|intensidad|ocupacion|carga|vmed|error|periodo_integracion|
+----+-------------------+---------+----------+---------+-----+----+-----+-------------------+
|1001|2024-01-01 00:00:00|      C30|       300|      1.0|    0|57.0|    N|                  5|
|1001|2024-01-01 00:15:00|      C30|       168|      0.0|    0|62.0|    N|                  5|
|1001|2024-01-01 00:30:00|      C30|       540|      2.0|    0|61.0|    N|                  5|
+----+-------------------+---------+----------+---------+-----+----+-----+-------------------+
only showing top 3 rows


+---+------+
| id|ciudad|
+---+------+
|  1|Málaga|
|  2|Madrid|
|  3| Cádiz|
+---+------+

Spark Context Master: spark://master:7077
Resultados:
[('es', 1), ('docker', 1), ('esto', 1), ('hola', 1), ('spark', 1), ('con', 1), ('una', 1), ('mundo', 1), ('prueba', 1)]


'4.0.0'

In [6]:
spark

## Prueba de MINST con torchdistributor

In [7]:
## Este ejemplo está tomado de github. 

from pyspark.sql import SparkSession
from pyspark.ml.torch.distributor import TorchDistributor
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, DistributedSampler
import os
import torch.distributed as dist

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


def main():
    dist.init_process_group(backend="gloo")

    rank = dist.get_rank()
    world_size = dist.get_world_size()

    torch.manual_seed(0)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('/tmp/mnist', train=True, download=True, transform=transform)
    sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)

    model = Net()
    model = nn.parallel.DistributedDataParallel(model)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, 20):  # solo 2 epochs para prueba
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0 and rank == 0:
                print(f"[Epoch {epoch}] Batch {batch_idx} Loss: {loss.item()}")
    dist.destroy_process_group()

distributor = TorchDistributor(
    #num_processes=3,  # spark.sparkContext.defaultParallelism
    num_processes=spark.sparkContext.defaultParallelism,
    local_mode = False,
    use_gpu=False)

distributor.run(main)



Started distributed training with 10 executor processes
[Epoch 1] Batch 0 Loss: 2.3251395225524902                        (0 + 10) / 10]
[Epoch 2] Batch 0 Loss: 0.20318341255187988
[Epoch 3] Batch 0 Loss: 0.15609514713287354
[Epoch 4] Batch 0 Loss: 0.12071896344423294
[Epoch 5] Batch 0 Loss: 0.09943787008523941
[Epoch 6] Batch 0 Loss: 0.08340492099523544
[Epoch 7] Batch 0 Loss: 0.07449784874916077
[Epoch 8] Batch 0 Loss: 0.06500767916440964
[Epoch 9] Batch 0 Loss: 0.05636418238282204
[Epoch 10] Batch 0 Loss: 0.045968152582645416
[Epoch 11] Batch 0 Loss: 0.03986917808651924
[Epoch 12] Batch 0 Loss: 0.035076674073934555
[Epoch 13] Batch 0 Loss: 0.02744406647980213
[Epoch 14] Batch 0 Loss: 0.027873048558831215
[Epoch 15] Batch 0 Loss: 0.028492441400885582
[Epoch 16] Batch 0 Loss: 0.02563989907503128
[Epoch 17] Batch 0 Loss: 0.024664783850312233
[Epoch 18] Batch 0 Loss: 0.01674218475818634
[Epoch 19] Batch 0 Loss: 0.018313443288207054
Finished distributed training with 10 executor processe

In [8]:
!ls /mnt/volume/puntos

101.csv  137.csv  173.csv  209.csv  242.csv  37.csv  71.csv
104.csv  140.csv  176.csv  212.csv  245.csv  40.csv  74.csv
107.csv  143.csv  179.csv  215.csv  248.csv  43.csv  77.csv
110.csv  146.csv  182.csv  218.csv  251.csv  46.csv  80.csv
113.csv  149.csv  185.csv  221.csv  254.csv  49.csv  83.csv
116.csv  152.csv  188.csv  224.csv  257.csv  52.csv  86.csv
119.csv  155.csv  191.csv  227.csv  25.csv   55.csv  89.csv
122.csv  158.csv  194.csv  22.csv   260.csv  58.csv  92.csv
126.csv  161.csv  197.csv  230.csv  263.csv  61.csv  95.csv
128.csv  164.csv  200.csv  233.csv  28.csv   64.csv  98.csv
131.csv  167.csv  203.csv  236.csv  31.csv   67.csv  festivos2025.csv
134.csv  170.csv  206.csv  239.csv  34.csv   68.csv  historico_festivos.csv


In [9]:
import os
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("tipo_elem", StringType(), True),
    StructField("distrito", IntegerType(), True),
    StructField("id", IntegerType(), True),
    StructField("cod_cent", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("utm_x", DoubleType(), True),
    StructField("utm_y", DoubleType(), True),
    StructField("longitud", DoubleType(), True),
    StructField("latitud", DoubleType(), True)
])

acumulado = None

li = os.listdir("/mnt/volume/puntos")

for q in range(1, 400, 1):
    file_path = "/mnt/volume/puntos/" + str(q) + ".csv"
    if str(q) + '.csv' in li:
      nuevo_df = spark.read.format("csv") \
            .option("sep", ";") \
            .option("header", "true") \
            .schema(schema) \
            .load('file://' + file_path)
      if acumulado is None:
        acumulado = nuevo_df
      else:
        existentes = acumulado.join(nuevo_df, on="id", how="left_anti")
        acumulado = existentes.unionByName(nuevo_df)

In [10]:
acumulado.show()
print(acumulado.count())

25/09/08 20:25:19 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
25/09/08 20:25:19 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
25/09/08 20:25:21 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
                                                                                

+-----+---------+--------+--------+--------------------+----------------+----------------+-----------------+----------------+
|   id|tipo_elem|distrito|cod_cent|              nombre|           utm_x|           utm_y|         longitud|         latitud|
+-----+---------+--------+--------+--------------------+----------------+----------------+-----------------+----------------+
|10339|      URB|       9|   17034|Quintana E-O - Ma...|439184.091811248|4475516.01939195|-3.71696021836144|40.4280582423154|
| 6504|      URB|      20|   71039|Av. Jaca - Av. Ja...|            NULL|            NULL|             NULL|            NULL|
| 7132|      URB|       4|    6016|(AFOROS) Velazque...|441954.508259539|4475053.14160329|-3.68425971438312|40.4240863309414|
| 4300|      URB|       1|   16025|Mesoneros Romanos...|440247.624601693|4474560.73099691|-3.70433334217253|40.4195296363783|
| 3766|      URB|       9|   31061|(TACTICO) Isla de...|438925.304560274|4479395.44576109|-3.72038386410503| 40.462986

25/09/08 20:25:35 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
[Stage 174:==================================================>    (75 + 7) / 82]

54633


In [11]:
acumulado = acumulado.where('id is not NULL')
acumulado.createOrReplaceTempView("qqq")
spark.sql('select * from qqq where longitud is null or latitud is null or id is null').show()

25/09/08 20:25:53 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
25/09/08 20:25:54 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
25/09/08 20:25:54 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
25/09/08 20:25:55 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB


+----+---------+--------+--------+--------------------+-----+-----+--------+-------+
|  id|tipo_elem|distrito|cod_cent|              nombre|utm_x|utm_y|longitud|latitud|
+----+---------+--------+--------+--------------------+-----+-----+--------+-------+
|6504|      URB|      20|   71039|Av. Jaca - Av. Ja...| NULL| NULL|    NULL|   NULL|
+----+---------+--------+--------+--------------------+-----+-----+--------+-------+



In [12]:
df_puntos = acumulado
df_puntos.createOrReplaceTempView("qqq")
spark.sql('select id,count(*) from qqq where id is not null group by id having count(*)>1').show()

25/09/08 20:26:09 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
25/09/08 20:26:13 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
                                                                                

+---+--------+
| id|count(1)|
+---+--------+
+---+--------+



In [13]:
df_puntos = spark.sql('select id,longitud, latitud from qqq where id is not null and longitud is not null and latitud is not null')

In [14]:
spark.sql('select count (*) from qqq').show()

df_puntos.count()

25/09/08 20:26:26 WARN DAGScheduler: Broadcasting large task binary with size 4.4 MiB
                                                                                

+--------+
|count(1)|
+--------+
|    5174|
+--------+



25/09/08 20:26:40 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
                                                                                

5173

In [15]:
import urllib.request

url_festivos = 'https://datos.comunidad.madrid/catalogo/dataset/02c712b5-5009-4ffb-b388-3cfb4fca207d/resource/02d9ad68-6614-4105-a0e7-0a3c429859cf/download/festivos_regionales_historicos.csv'
url_2025 = 'https://datos.comunidad.madrid/dataset/f160eb6c-6715-471e-9bc0-38497aae950f/resource/eb9fe481-076d-48bb-bfe4-0df0e301f765/download/festivos_regionales.csv'

output_file = "/mnt/volume/puntos/historico_festivos.csv"
output_file25 = "/mnt/volume/puntos/festivos2025.csv"
urllib.request.urlretrieve(url_festivos, output_file)

df_festivos = spark.read.format("csv").option("inferSchema","true").option("sep",";").option("header","true") \
    .load("file:///mnt/volume/puntos/historico_festivos.csv")

urllib.request.urlretrieve(url_2025, output_file25)
df_festivos2 = spark.read.format("csv").option("inferSchema","true").option("sep",";").option("header","true") \
    .load("file:///mnt/volume/puntos/festivos2025.csv")

df_festivos = df_festivos.union(df_festivos2)

df_festivos.printSchema()



root
 |-- a�o: integer (nullable = true)
 |-- fecha_festivo: date (nullable = true)
 |-- festividad: string (nullable = true)



In [16]:
df_festivos.show()

+----+-------------+--------------------+
| a�o|fecha_festivo|          festividad|
+----+-------------+--------------------+
|1998|   1998-01-01|           A�o Nuevo|
|1998|   1998-01-06|  Epifan�a del Se�or|
|1998|   1998-03-19|            San Jos�|
|1998|   1998-04-09|        Jueves Santo|
|1998|   1998-04-10|       Viernes Santo|
|1998|   1998-05-01|  Fiesta del Trabajo|
|1998|   1998-05-02|Fiesta de la Comu...|
|1998|   1998-08-15|Asunci�n de la Vi...|
|1998|   1998-10-12|Fiesta Nacional d...|
|1998|   1998-11-02|    Todos los Santos|
|1998|   1998-12-08|Inmaculada Concep...|
|1998|   1998-12-25| Natividad del Se�or|
|1999|   1999-01-01|           A�o Nuevo|
|1999|   1999-01-06|  Epifan�a del Se�or|
|1999|   1999-03-19|            San Jos�|
|1999|   1999-04-01|        Jueves Santo|
|1999|   1999-04-02|       Viernes Santo|
|1999|   1999-05-01|  Fiesta del Trabajo|
|1999|   1999-05-15|          San Isidro|
|1999|   1999-08-16|Asunci�n de la Vi...|
+----+-------------+--------------

In [17]:
from pyspark.sql.functions import date_format
df_festivos = df_festivos.select(date_format('fecha_festivo','yyyy-MM-dd').alias('fecha_festivo')).distinct()
df_festivos.show()

+-------------+
|fecha_festivo|
+-------------+
|   2000-04-20|
|   2000-04-21|
|   2000-05-15|
|   2001-01-01|
|   2001-05-01|
|   2001-05-02|
|   2001-05-15|
|   2002-03-29|
|   2002-05-01|
|   2003-01-01|
|   2004-01-01|
|   2005-12-25|
|   2006-01-01|
|   2006-04-14|
|   2006-05-01|
|   2006-12-08|
|   2008-07-25|
|   2009-01-01|
|   2009-01-06|
|   2009-05-02|
+-------------+
only showing top 20 rows


In [18]:
df_festivos.write.format("parquet").mode("overwrite").save("hdfs://master:9000/festivos")

In [19]:
df_puntos.write.format("parquet").mode("overwrite").save("hdfs://master:9000/puntos")

25/09/08 20:26:52 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
                                                                                

In [20]:
df_puntos.describe().show()
df_festivos.createOrReplaceTempView("tb_festivos")
df_puntos.createOrReplaceTempView("tb_puntos")

25/09/08 20:27:07 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
[Stage 688:======================================>               (59 + 10) / 82]

+-------+-----------------+--------------------+--------------------+
|summary|               id|            longitud|             latitud|
+-------+-----------------+--------------------+--------------------+
|  count|             5173|                5173|                5173|
|   mean|6832.655325729751|  -3.684087021639414|   40.43038444497566|
| stddev|2692.607578324742|0.042889279891849236|0.038829711673342614|
|    min|                0|   -3.83688562997397|   40.33245364209287|
|    max|            11451|   -3.55162282763333|   40.51561092557125|
+-------+-----------------+--------------------+--------------------+



## En este punto tenemos en df_puntos los puntos de medida y en df_festivos los días festivos. A continuación, leemos el dataset. También los hemos grabado en hdfs.

In [21]:
from pyspark.sql.types import TimestampType


'''root
 |-- id: integer (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- tipo_elem: string (nullable = true)
 |-- intensidad: integer (nullable = true)
 |-- ocupacion: double (nullable = true)
 |-- carga: integer (nullable = true)
 |-- vmed: double (nullable = true)
 |-- error: string (nullable = true)
 |-- periodo_integracion: integer (nullable = true)
'''

## Esquema del conjunto de datos

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("fecha", TimestampType(), True),
    StructField("tipo_elem", StringType(), True),
    StructField("intensidad", IntegerType(), True),
    StructField("ocupacion", DoubleType(), True),
    StructField("carga", IntegerType(), True),
    StructField("vmed", DoubleType(), True),
    StructField("error", StringType(), True),
    StructField("periodo_integracion", IntegerType(), True)
])



In [22]:
from pyspark.sql.functions import col, to_timestamp, to_char,year, month, dayofmonth, hour, minute, dayofweek, weekofyear, quarter, expr, split, lpad, concat_ws, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark.sql.functions import to_date

def Preprocesa(my_df):    
    my_df = my_df.withColumn("year", year(col("fecha"))) \
      .withColumn("month", month(col("fecha"))) \
      .withColumn("day", dayofmonth(col("fecha"))) \
      .withColumn("hour", hour(col("fecha"))) \
      .withColumn("min", minute(col("fecha"))) \
      .withColumn("day_of_week", expr("((dayofweek(fecha) + 5) % 7) + 1")) \
      .withColumn("week_of_year", weekofyear(col("fecha"))) \
      .withColumn("num_min", hour(col("fecha")) * 60 + minute(col("fecha"))) \
      .withColumn("quarter", quarter(col("fecha")))       
    my_df = my_df.withColumn("tipo_elem", when(col("tipo_elem") == "M30", "C30").otherwise(col("tipo_elem")))
    my_df.createOrReplaceTempView("tb_temp")    
    joined_df = spark.sql('SELECT m.*,p.longitud,p.latitud from tb_puntos p, tb_temp m WHERE m.id = p.id')
    my_df = joined_df    
    my_df = my_df.where("error = 'N'")
    my_df = my_df.dropna().dropDuplicates().drop('id')
    my_df.createOrReplaceTempView("tb_temp")    
    my_df = spark.sql('''SELECT m.*,'S' as festivo from tb_temp m
        WHERE date_format(fecha,"yyyy-MM-dd") in (select date_format(fecha_festivo,"yyyy-MM-dd")  from tb_festivos)
        union
        SELECT m.*,'N' as festivo from tb_temp m 
        WHERE date_format(fecha,"yyyy-MM-dd") not in (select date_format(fecha_festivo,"yyyy-MM-dd") from tb_festivos)''')      
    my_df = my_df.withColumn('fecha',date_format('fecha','yyyy-MM-dd'))    
    my_df = my_df.drop('vmed').drop('error').drop('periodo_integracion').where('intensidad >= 0 and carga >=0')    
    return my_df



def PreprocesaFechasErroneas(my_df):
    split_datetime = split(col("fecha"), " ")
    date_part_str = split_datetime.getItem(0)
    time_part_str = split_datetime.getItem(1)
    split_date = split(date_part_str, "/")
    day_part = split_date.getItem(0)
    month_part = split_date.getItem(1)
    year_part = split_date.getItem(2)
    day_part_padded = lpad(day_part, 2, "0")
    month_part_padded = lpad(month_part, 2, "0")
    padded_date_part = concat_ws("/", day_part_padded, month_part_padded, year_part)
    time_split = split(time_part_str, ":")
    hour_part = time_split.getItem(0)
    minute_part = time_split.getItem(1)
    hour_part_padded = lpad(hour_part, 2, "0")
    cleaned_fecha_string = concat_ws(" ", padded_date_part, concat_ws(":", hour_part_padded, minute_part))
    my_df = my_df.withColumn("fecha_timestamp", to_timestamp(cleaned_fecha_string, "dd/MM/yy HH:mm"))   
    my_df = my_df.withColumn("tipo_elem", when(col("tipo_elem") == "M30", "C30").otherwise(col("tipo_elem")))
    my_df = my_df.withColumn("year", year(col("fecha_timestamp"))) \
      .withColumn("month", month(col("fecha_timestamp"))) \
      .withColumn("day", dayofmonth(col("fecha_timestamp"))) \
      .withColumn("hour", hour(col("fecha_timestamp"))) \
      .withColumn("min", minute(col("fecha_timestamp"))) \
      .withColumn("day_of_week", expr("((dayofweek(fecha_timestamp) + 5) % 7) + 1")) \
      .withColumn("week_of_year", weekofyear(col("fecha_timestamp"))) \
      .withColumn("num_min", hour(col("fecha_timestamp")) * 60 + minute(col("fecha_timestamp"))) \
      .withColumn("quarter", quarter(col("fecha_timestamp")))          
    my_df = my_df.withColumn('fecha',date_format('fecha_timestamp','yyyy-MM-dd'))
    my_df.createOrReplaceTempView("tb_temp")    
    joined_df = spark.sql('SELECT m.*,p.longitud,p.latitud from tb_puntos p, tb_temp m WHERE m.id = p.id')
    my_df = joined_df    
    my_df = my_df.where("error = 'N'")
    my_df = my_df.dropna().dropDuplicates().drop('id')
    my_df.createOrReplaceTempView("tb_temp")    
    my_df = spark.sql('''SELECT m.*,'S' as festivo from tb_temp m
        WHERE fecha in (select date_format(fecha_festivo,"yyyy-MM-dd")  from tb_festivos)
        union
        SELECT m.*,'N' as festivo from tb_temp m 
        WHERE fecha not in (select date_format(fecha_festivo,"yyyy-MM-dd") from tb_festivos)''')        
    my_df = my_df.drop('id').drop('fec').drop('fecha_timestamp').drop('vmed').drop('error').drop('periodo_integracion').where('intensidad >= 0 and carga >=0')    
    return my_df

def LeerMes(mm, aaaa):
    archivo = '/mnt/volume/cvs_ok/' + aaaa + '/' + mm + '-' + aaaa + '.csv'
    print (archivo)
    temp =  spark.read.format("csv").option("inferSchema","true").option("sep",";").option("header","true").schema(schema).load('file://' + archivo)    
    temp = Preprocesa(temp)   
    return temp

In [23]:
# De 2025, tenemos datos hasta julio.

def LeerAAAA(aaaa):
    acumulado = None  
    top = 13
    if  aaaa == '2025':
        top = 8
    for r in range(1,top):
        nn = f"{r:02d}"  
        if r == 1:
            acumulado = LeerMes(nn,aaaa)
        else:
            try:
                k = LeerMes(nn,aaaa)
                acumulado = acumulado.union(k)
            except Exception as e:
                print(f"Error al leer archivo {nn} {aaaa}': {e}")
        print (f'{aaaa} {nn}')        
    df_puntos.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/{aaaa}")
    return acumulado


In [24]:
my_df = LeerMes('01','2025')     
my_df.show()

/mnt/volume/cvs_ok/2025/01-2025.csv


25/09/08 20:27:47 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 20:27:53 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 20:28:00 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
                                                                                

+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|year|month|day|hour|min|day_of_week|week_of_year|num_min|quarter|        longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+----------------+-----------------+-------+
|2025-01-02|      C30|        60|      1.0|    5|2025|    1|  2|   0| 45|          4|           1|     45|      1|-3.6723088411066|40.48483856745793|      N|
|2025-01-02|      C30|        32|      1.0|    4|2025|    1|  2|   1| 30|          4|           1|     90|      1|-3.6723088411066|40.48483856745793|      N|
|2025-01-02|      C30|        12|      0.0|    2|2025|    1|  2|   4|  0|          4|           1|    240|      1|-3.6723088411066|40.48483856745793|      N|
|2025-01-02|      C30|        24|      0.0|    2|202

In [25]:
my_df = my_df.union(LeerMes('02','2025'))
my_df.show()

/mnt/volume/cvs_ok/2025/02-2025.csv


25/09/08 20:30:17 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 20:30:29 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 20:30:30 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:30:40 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
                                                                                

+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+-----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|year|month|day|hour|min|day_of_week|week_of_year|num_min|quarter|         longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+-----------------+-----------------+-------+
|2025-01-22|      URB|       288|      5.0|   12|2025|    1| 22|   9| 30|          3|           4|    570|      1|-3.67069838875071|40.41137964879736|      N|
|2025-01-22|      URB|       172|      2.0|    7|2025|    1| 22|   9| 45|          3|           4|    585|      1|-3.67069838875071|40.41137964879736|      N|
|2025-01-22|      URB|       332|      5.0|   14|2025|    1| 22|  10| 15|          3|           4|    615|      1|-3.67069838875071|40.41137964879736|      N|
|2025-01-22|      URB|       304|      4.0|   

In [26]:
my_df = my_df.union(LeerMes('03','2025'))
my_df.show()

/mnt/volume/cvs_ok/2025/03-2025.csv


25/09/08 20:35:32 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 20:35:52 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:35:52 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:35:53 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 20:36:08 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
                                                                                

+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+-----------------+-----------------+-------+
|     fecha|tipo_elem|intensidad|ocupacion|carga|year|month|day|hour|min|day_of_week|week_of_year|num_min|quarter|         longitud|          latitud|festivo|
+----------+---------+----------+---------+-----+----+-----+---+----+---+-----------+------------+-------+-------+-----------------+-----------------+-------+
|2025-01-01|      URB|       241|      2.0|   16|2025|    1|  1|   0| 45|          3|           1|     45|      1|-3.75316759460202|40.37240998155409|      S|
|2025-01-01|      URB|       124|      1.0|    9|2025|    1|  1|   3| 15|          3|           1|    195|      1|-3.75316759460202|40.37240998155409|      S|
|2025-01-01|      URB|        60|      0.0|    5|2025|    1|  1|   4| 15|          3|           1|    255|      1|-3.75316759460202|40.37240998155409|      S|
|2025-01-01|      URB|        72|      0.0|   

In [27]:
## Totales Q1 2025.

my_df.createOrReplaceTempView("datos_q1")    
spark.sql('SELECT count(*),year,month from datos_q1 group by year,month').show()
spark.sql('SELECT count(*) from datos_q1').show()


25/09/08 20:41:44 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 20:42:04 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:42:04 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:42:04 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 20:42:20 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
25/09/08 20:42:26 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
                                                                                

+--------+----+-----+
|count(1)|year|month|
+--------+----+-----+
|13181133|2025|    1|
|13264721|2025|    3|
|12012363|2025|    2|
+--------+----+-----+



25/09/08 20:47:34 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 20:47:54 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:47:54 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 20:47:54 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 20:48:08 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
                                                                                

+--------+
|count(1)|
+--------+
|38458217|
+--------+



In [28]:
# Esto realiza la lectura por años en batch. 

#my_df_2024 = LeerAAAA('2024')
#my_df_2025  = LeerAAAA('2025')
#my_df_1 = LeerMes('07','2025')                  


In [31]:
my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/2025Q1")

25/09/08 21:22:49 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 21:23:08 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:23:08 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 21:23:09 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:23:22 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB
                                                                                

In [32]:
my_df_2025 = my_df.union(LeerMes('04','2025'))
#my_df_2025.show()
my_df_2025 = my_df_2025.union(LeerMes('05','2025'))
#my_df_2025.show()
my_df_2025 = my_df_2025.union(LeerMes('06','2025'))
#my_df_2025.show()
my_df_2025 = my_df_2025.union(LeerMes('07','2025'))
#my_df_2025.show()

my_df_2025.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/2025")


/mnt/volume/cvs_ok/2025/04-2025.csv
/mnt/volume/cvs_ok/2025/05-2025.csv
/mnt/volume/cvs_ok/2025/06-2025.csv
/mnt/volume/cvs_ok/2025/07-2025.csv


25/09/08 21:38:05 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 21:38:37 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:37 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:37 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:37 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:38 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:38 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:38 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:44 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 21:38:47 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
25/09/08 21:38:51 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:38:54 WARN DAGScheduler: Broadcasting larg

In [33]:
my_df_2025.createOrReplaceTempView("datos_2025")    
spark.sql('SELECT count(*),year,month from datos_2025 group by year,month').show()
spark.sql('SELECT count(*) from datos_2025').show()

25/09/08 21:56:17 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 21:56:23 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:23 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:23 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:23 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:24 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 21:56:25 WARN DAGScheduler: Broadcasting larg

+--------+----+-----+
|count(1)|year|month|
+--------+----+-----+
|12012363|2025|    2|
|13440892|2025|    5|
|12943951|2025|    6|
|13499087|2025|    7|
|13181133|2025|    1|
|13264721|2025|    3|
|12058399|2025|    4|
+--------+----+-----+



25/09/08 22:14:20 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
25/09/08 22:14:26 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:26 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:26 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:26 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
25/09/08 22:14:27 WARN DAGScheduler: Broadcasting larg

+--------+
|count(1)|
+--------+
|90400546|
+--------+



In [34]:
# my_df : Q1 2025.
# my_df_2025: Año 2025
# my_df_aaaa : Datos desde hace un año (julio 24 / julio 25)


my_df_aaaa = my_df_2025
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('07','2024'))
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('08','2024'))
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('09','2024'))
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('10','2024'))
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('11','2024'))
#my_df_aaaa.show(3)
my_df_aaaa= my_df_aaaa.union(LeerMes('12','2024'))
#my_df_aaaa.show(3)

/mnt/volume/cvs_ok/2024/07-2024.csv
/mnt/volume/cvs_ok/2024/08-2024.csv
/mnt/volume/cvs_ok/2024/09-2024.csv
/mnt/volume/cvs_ok/2024/10-2024.csv
/mnt/volume/cvs_ok/2024/11-2024.csv
/mnt/volume/cvs_ok/2024/12-2024.csv


In [ ]:
my_df_aaaa.createOrReplaceTempView("datos_aaaa")    
spark.sql('SELECT count(*),year from datos_aaaa group by year').show()
spark.sql('SELECT count(*) from datos_aaaa').show()

In [ ]:
my_df_aaaa.count()

In [ ]:
my_df_aaaa.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/desdehace1a")

In [ ]:
my_df_q1 = my_df
df_mediciones = my_df
#df_mediciones = my_df_aaaa
#my_df = df_mediciones
my_df.summary().show()
#my_df_q1.summary().show()
#my_df_2025.summary().show()
#my_df_aaaa.summary().show()
#df_mediciones.summary().show()

In [ ]:
print ('Primer Q 2025', my_df.count())
print ('Primer Q 2025', df_mediciones.count())
print ('Primer Q 2025', my_df_q1.count())
print ('Año 2025', my_df_2025.count())
print ('Ultimo año', my_df_aaaa.count())

In [ ]:
print("DF creados:")
spark_dataframes = {name: obj for name, obj in globals().items() if isinstance(obj, type(spark.createDataFrame([], "struct<>")))}
for name, zz in spark_dataframes.items():
    print(f"- {name}: {zz}")

print("\nVistas temporales:")
for view_name in spark.catalog.listTables():
    if view_name.isTemporary:
        print(f"- {view_name.name}")


In [ ]:
'''df = None
zz = None
acumulado = None
nuevo_df = None
existentes = None
af = None
tb_festivos = None
joined_df = None
tempo = None
result = None

spark.catalog.dropTempView("qqq")
spark.catalog.dropTempView("tb_puntos")
spark.catalog.dropTempView("tb_medidas")
spark.catalog.dropTempView("tb_festivos")
spark.catalog.dropTempView("r")
spark.catalog.dropTempView("tb_temp")'''

## Análisis exploratorio de datos. 
### En este punto, tenemos eliminados, valores nulos y medidas erróneas. Sólo tenemos dos columnas categóricas, tipo_elem y festivo. Los datos los tenemos en df_mediciones (último año). Los pasamos a my_df. Vamos a hacer el estudio sobre el último año (de julio de 2024 a julio de 2025). 

In [ ]:
df_mediciones.xdescribe().show()

In [39]:
my_df.describe().show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql.functions import col, sum

print("Comprobamos valores nulos (no debería haberlos):")
my_df.select([sum(col(c).isNull().cast("integer")).alias(c) for c in my_df.columns]).show()

## Estudio de las columnas

In [ ]:
from pyspark.sql.functions import col, count, mean, stddev, corr, percentile_approx

numerical_cols = [col for col, dtype in my_df.dtypes if dtype != 'string' and dtype != 'timestamp' and name != 'festivo'] 

print("\nCorrelación de columnas numéricas con 'intensidad':")
num_cols_cor = [col for col in numerical_cols if col != 'intensidad']

print ("Columnas {num_cols_cor}")

correlaciones = []
for col_name in num_cols_cor:
    try:
        corr = my_df.stat.corr(col_name, 'intensidad')
        print(f"Correlación entre '{col_name}' y 'intensidad': {corr}")
        correlaciones.append((col_name, corr))
    except Exception as e:
        print(f"No se pudo calcular la correlación para '{col_name}': {e}")


In [ ]:

print ("Correlaciones: ",correlaciones)


for a in correlaciones:    
    print(a[0],a[1])
    
print("\nDescripción de las no numéricas:")
categorical_cols = ['tipo_elem','festivo']


for col_name in categorical_cols:
    print(f"\nRecuento de valores '{col_name}':")
    my_df.groupBy(col_name).count().orderBy("count", ascending=False).show()


In [ ]:
from pyspark.sql.functions import mean, stddev, col, percentile_approx

print("Estadísticas 'intensidad':")
my_df.select(
    mean("intensidad").alias("mean_intensidad"),
    stddev("intensidad").alias("stddev_intensidad"),
    percentile_approx("intensidad", 0.5, 100).alias("median_intensidad") 
).show()


In [ ]:
from pyspark.sql.functions import col, count, mean, stddev, corr, percentile_approx

print("Distribución de las columas de categorías:")
categorical_cols = [col_name for col_name, dtype in my_df.dtypes if dtype == 'string']

print("\nDistribución de las columnas numéricas:")
numerical_cols = [col_name for col_name, dtype in my_df.dtypes if dtype in ['int', 'double', 'float', 'long']]
numerical_cols.remove('intensidad')
my_df.select(numerical_cols).describe().show()

print("\nMedia y mediana de 'intensidad':")
for col_name in categorical_cols:
    print(f"\nDatos para '{col_name}':")
    my_df.groupBy(col_name).agg(
        mean("intensidad").alias("mean_intensidad"),
        percentile_approx("intensidad", 0.5, 100).alias("median_intensidad")
    ).orderBy(col_name).show()



In [ ]:
for t in correlaciones:
  print ('Correlación de ' +  t[0] + '      :', t[1])

print("\nDescripción de las no numéricas:")
categorical_cols = ['tipo_elem','festivo']
my_df.select(categorical_cols).describe().show()

In [ ]:
### Prueba
# my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/2025Q1")
#my_df = spark.read.parquet("hdfs://master:9000/medidas/2025Q1")


In [ ]:
my_df.cache()
my_df.createOrReplaceTempView("my_df")
spark.sql('select count(*) from my_df').show()

In [ ]:
my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/medidas/2025Q1_integer")

In [ ]:
my_df.createOrReplaceTempView("tabla_intensidad")    
q = spark.sql('select latitud, longitud, sum(intensidad) as suma,avg(intensidad) as media from tabla_intensidad group by latitud, longitud')    
q.show(3)

In [ ]:
r = q.toPandas()

r.head()

In [ ]:
import matplotlib.pyplot as plt

r.plot(kind="scatter", x="longitud", y="latitud", grid=True,
             s=r["suma"] / 200000, label="intensidad",
             c="suma", cmap="jet", colorbar=True,
             legend=True, sharex=False, figsize=(10, 7))

r.plot(kind="scatter", x="longitud", y="latitud", grid=True,
             s=r["media"] / 500, label="intensidad",
             c="media", cmap="jet", colorbar=True,
             legend=True, sharex=False, figsize=(10, 7))

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

my_df.printSchema()
columns_with_types = my_df.dtypes
integer_cols = [name for name, dtype in columns_with_types if dtype == 'int']

for col_name in integer_cols:
    my_df = my_df.withColumn(col_name, col(col_name).cast(DoubleType()))

my_df.printSchema()

In [ ]:
# Grabamos los ficheros de datos:

my_df.cache()
my_df.write.format("parquet").mode("overwrite").save("hdfs://master:9000/my_df_double")
